In [146]:
# Dependencies
import requests
import json
import quandl
import pandas as pd
from datetime import datetime
import plotly.express as px



In [147]:
date_df = pd.read_csv('../../db/sentences_sample_with_datetime2.csv',sep=',')
date_df['call_date'] = pd.to_datetime(date_df['call_date']).dt.date
date_df['base_date'] = pd.to_datetime(date_df['base_date']).dt.date
date_df['next_date'] = pd.to_datetime(date_df['next_date']).dt.date

date_df.head()



,call_datetime,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment,call_date,call_time,unique_id,base_date,next_date
0,2019-11-01 10:00:00,457,457,NaN,Good morning and welcome to the Dominion Ener...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.7906,2019-11-01,10:00:00,457D,2019-10-31,2019-11-01
1,2019-11-01 10:00:00,458,458,NaN,"At this time, each of your lines is in a liste...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,2019-11-01,10:00:00,458D,2019-10-31,2019-11-01
2,2019-11-01 10:00:00,459,459,NaN,"At the conclusion of today’s presentation, we ...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,2019-11-01,10:00:00,459D,2019-10-31,2019-11-01
3,2019-11-01 10:00:00,460,460,NaN,[Operator Instructions] I would now like to tu...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.3612,2019-11-01,10:00:00,460D,2019-10-31,2019-11-01
4,2019-11-01 10:00:00,461,461,NaN,Good morning and welcome.,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.7096,2019-11-01,10:00:00,461D,2019-10-31,2019-11-01


In [148]:
price_df = pd.read_csv('../../db/ticker_closing_prices.csv',sep=',')
price_df['Date'] = pd.to_datetime(price_df['Date']).dt.date
price_df.head()


,Date,High,Low,Open,Close,Volume,Adj Close,ticker
0,2019-01-02,165.139999,160.949997,161.210007,164.080002,1465400.0,164.080002,VRTX
1,2019-01-03,169.320007,163.179993,165.830002,163.729996,1690900.0,163.729996,VRTX
2,2019-01-04,173.309998,164.899994,165.899994,172.699997,1874500.0,172.699997,VRTX
3,2019-01-07,176.779999,170.869995,173.050003,175.850006,1294600.0,175.850006,VRTX
4,2019-01-08,178.410004,174.720001,177.000000,178.190002,1653200.0,178.190002,VRTX


In [153]:
merged_df = date_df
merged_df.count()

call_datetime      45876
Unnamed: 0         45876
id                 45876
sentiment              0
sentence           45876
ticker             45876
call_title         45876
speaker            45876
call_section       45876
vader_sentiment    45876
call_date          45876
call_time          45876
unique_id          45876
base_date          45876
next_date          45876
dtype: int64

In [154]:
merged_df = date_df.merge(price_df, left_on=['ticker','base_date'], right_on = ['ticker','Date'], how='inner')
merged_df = merged_df.drop(columns=['Date', 'High','Low','Open','Adj Close'])
merged_df = merged_df.rename(columns={"Close": "base_close", "Volume": "base_volume"})
merged_df.count()

call_datetime      46089
Unnamed: 0         46089
id                 46089
sentiment              0
sentence           46089
ticker             46089
call_title         46089
speaker            46089
call_section       46089
vader_sentiment    46089
call_date          46089
call_time          46089
unique_id          46089
base_date          46089
next_date          46089
base_close         46089
base_volume        46089
dtype: int64

In [155]:
merged_df = merged_df.merge(price_df, left_on=['ticker','next_date'], right_on = ['ticker','Date'], how='inner')
merged_df = merged_df.drop(columns=['Date', 'High','Low','Open','Adj Close'])
merged_df = merged_df.rename(columns={"Close": "next_close", "Volume": "next_volume"})
merged_df.count()

call_datetime      52385
Unnamed: 0         52385
id                 52385
sentiment              0
sentence           52385
ticker             52385
call_title         52385
speaker            52385
call_section       52385
vader_sentiment    52385
call_date          52385
call_time          52385
unique_id          52385
base_date          52385
next_date          52385
base_close         52385
base_volume        52385
next_close         52385
next_volume        52385
dtype: int64

In [156]:
mkt_cap_df = pd.read_csv('../../db/ticker_mktCap_test.csv',sep=',')
merged_df = merged_df.merge(mkt_cap_df, left_on=['ticker'], right_on = ['ticker'], how='left')
merged_df = merged_df.drop_duplicates(subset=['unique_id'])
merged_df


,call_datetime,Unnamed: 0_x,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment,...,call_time,unique_id,base_date,next_date,base_close,base_volume,next_close,next_volume,Unnamed: 0_y,marketCap
0,2019-11-01 10:00:00,457,457,NaN,Good morning and welcome to the Dominion Ener...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.7906,...,10:00:00,457D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
1,2019-11-01 10:00:00,458,458,NaN,"At this time, each of your lines is in a liste...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,...,10:00:00,458D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
2,2019-11-01 10:00:00,459,459,NaN,"At the conclusion of today’s presentation, we ...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,...,10:00:00,459D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
3,2019-11-01 10:00:00,460,460,NaN,[Operator Instructions] I would now like to tu...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.3612,...,10:00:00,460D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
4,2019-11-01 10:00:00,461,461,NaN,Good morning and welcome.,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.7096,...,10:00:00,461D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
5,2019-11-01 10:00:00,462,462,NaN,I encourage you to visit our Investor Relation...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.6369,...,10:00:00,462D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
6,2019-11-01 10:00:00,463,463,NaN,The Investor Relations team will be available ...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.0000,...,10:00:00,463D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
7,2019-11-01 10:00:00,464,464,NaN,Earnings materials including our prepared rema...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,-0.3818,...,10:00:00,464D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
8,2019-11-01 10:00:00,465,465,NaN,"Please refer to our SEC filings, including our...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.3182,...,10:00:00,465D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936
9,2019-11-01 10:00:00,466,466,NaN,"This morning, we will discuss some measures of...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.0000,...,10:00:00,466D,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936


In [157]:
merged_df['percent_change_price'] = ((merged_df['next_close']/merged_df['base_close'])-1)*100
merged_df['percent_change_volume'] = ((merged_df['next_volume']/merged_df['base_volume'])-1)*100

merged_df

,call_datetime,Unnamed: 0_x,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment,...,base_date,next_date,base_close,base_volume,next_close,next_volume,Unnamed: 0_y,marketCap,percent_change_price,percent_change_volume
0,2019-11-01 10:00:00,457,457,NaN,Good morning and welcome to the Dominion Ener...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.7906,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
1,2019-11-01 10:00:00,458,458,NaN,"At this time, each of your lines is in a liste...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
2,2019-11-01 10:00:00,459,459,NaN,"At the conclusion of today’s presentation, we ...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
3,2019-11-01 10:00:00,460,460,NaN,[Operator Instructions] I would now like to tu...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.3612,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
4,2019-11-01 10:00:00,461,461,NaN,Good morning and welcome.,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.7096,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
5,2019-11-01 10:00:00,462,462,NaN,I encourage you to visit our Investor Relation...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.6369,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
6,2019-11-01 10:00:00,463,463,NaN,The Investor Relations team will be available ...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.0000,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
7,2019-11-01 10:00:00,464,464,NaN,Earnings materials including our prepared rema...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,-0.3818,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
8,2019-11-01 10:00:00,465,465,NaN,"Please refer to our SEC filings, including our...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.3182,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076
9,2019-11-01 10:00:00,466,466,NaN,"This morning, we will discuss some measures of...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.0000,...,2019-10-31,2019-11-01,82.550003,2424000.0,83.269997,3169600.0,D,67584167936,0.872191,30.759076


In [158]:
vader_group = merged_df.groupby(['call_section'])
vader_bars = pd.DataFrame(vader_group['vader_sentiment'].mean())
vader_bars

,vader_sentiment
call_section,
answer,0.250983
operator_instruction,0.151514
presentation,0.304248
question,0.242807


In [159]:
group_df = merged_df.loc[merged_df['call_section']=='presentation']
group_df = group_df.groupby(['ticker'])
group_sent_avg = group_df['vader_sentiment'].mean()
group_price_avg = group_df['percent_change_price'].mean()
group_vol_avg = abs(group_df['percent_change_volume'].mean())


In [160]:
fig = px.scatter(x=group_sent_avg, y=group_price_avg, trendline="ols", size=group_vol_avg)
fig.show()



In [161]:
vader_bars=vader_bars.reset_index()
fig2 = px.bar(vader_bars,x='call_section', y='vader_sentiment')
fig2.show()


In [162]:
merged_df.to_csv('../../db/complete_merged_dataset.csv', sep=',')